In [1]:
!pip install langchain_community langchain langchain-groq langchain-mistralai langchain-together GitPython -qU

# Github Loader

In [2]:
# from langchain_community.document_loaders import GitLoader

In [3]:
# loader = GitLoader(clone_url="https://github.com/10mudassir007/grocery-store-crud-app",
#                    repo_path="/test_repo/", branch='main')

In [4]:
# data = loader.load()

# RAG

In [5]:
!pip install opik ragas langchain-huggingface faiss-cpu -qU

In [6]:
import os
from google.colab import userdata

os.environ['GROQ_API_KEY'] = userdata.get("GROQ_API_KEY")
#os.environ['OPIK_API_KEY'] = userdata.get("OPIK_API_KEY")


In [7]:
from langchain_community.document_loaders import WebBaseLoader

docs = ["https://medium.com/@muddassir10/optimizing-response-accuracy-in-rag-with-reranking-56012090a9f7","https://medium.com/@muddassir10/ai-driven-plant-disease-diagnosis-enhancing-agricultural-efficiency-2ada46e8729a"]

data = WebBaseLoader(docs).load()

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500,chunk_overlap=30)

splitted_docs = splitter.split_documents(data)

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

vector_db = FAISS.from_documents(documents=splitted_docs,
                                 embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
                                 docstore=InMemoryDocstore())

In [10]:
retriever = vector_db.as_retriever()

In [11]:
from langchain import hub

prompt = hub.pull('rlm/rag-prompt')
#prompt[0].prompt.template = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Provide the output in JSON format,\n Use the function provided if necessary\nQuestion: {question} \nContext: {context} \nAnswer:"

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2)

In [13]:
from langchain_core.output_parsers import JsonOutputParser
llm_chain = prompt | llm

In [14]:
query = "What are the projects listed in the documents"
response = llm_chain.invoke({"question":query,"context":retriever.invoke(query)})

In [15]:
response

AIMessage(content='The projects listed in the documents include RAG (Retrieval-Augmented Generation) with reranking for improving response accuracy, and an AI-powered plant disease detection system using computer vision and deep learning. These projects aim to enhance language processing and agricultural efficiency, respectively.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 1826, 'total_tokens': 1880, 'completion_time': 0.271936397, 'prompt_time': 0.332775273, 'queue_time': 0.019774225000000034, 'total_time': 0.60471167}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3884478861', 'finish_reason': 'stop', 'logprobs': None}, id='run-260fe063-2d3a-44a3-a5d7-7166fb69fa73-0', usage_metadata={'input_tokens': 1826, 'output_tokens': 54, 'total_tokens': 1880})

## OPIK EVALUATION

In [ ]:
import opik

opik.configure(use_local=False)

OPIK: Opik is already configured. You can check the settings by viewing the config file at /root/.opik.config


In [22]:
os.environ["OPIK_PROJECT_NAME"] = "test_opik"
os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")
os.environ['MISTRAL_API_KEY'] = userdata.get("MISTRAL_API_KEY")

In [24]:
from opik.integrations.langchain import OpikTracer

opik_tracer = OpikTracer(tags=["simple_chain"])

OPIK: ========================
The workspace and API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.



In [25]:
from opik import Opik, track
from opik.evaluation import evaluate
from opik.evaluation.metrics import AnswerRelevance,Hallucination
from opik.integrations.langchain import OpikTracer

# Initialize the tracer
opik_tracer = OpikTracer()

response = llm_chain.invoke({"question":query,"context":retriever.invoke(query)},callbacks=[opik_tracer])

OPIK: ========================
The workspace and API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.



In [26]:
from opik.evaluation.metrics import Hallucination
retrieved_docs = [str(x) for x in retriever.invoke(query)]
hallucination_metric = Hallucination(model="mistral/mistral-tiny")
result = hallucination_metric.score(
    input=query,
    output=str(llm_chain.invoke({"question":query,"context":retrieved_docs})),
    context=retrieved_docs,
)
print(result.value)
print(result.reason)

OPIK: ========================
The workspace and API key must be specified to log data to https://www.comet.com/opik.
You can use `opik configure` CLI command to configure your environment for logging.
See the configuration details in the docs: https://www.comet.com/docs/opik/tracing/sdk_configuration.

OPIK: Started logging traces to the "test_opik" project at https://www.comet.com/opik/default/redirect/projects?name=test_opik.



0.0
['The output provides a list of projects mentioned in the context, which is accurate and faithful to the provided context.', "The output does not introduce any new information beyond what's provided in the context.", 'The output does not contradict any information given in the context.', 'The output does not contradict well-established facts or general knowledge.', "The output correctly associates the projects with the right entity (the context), which is the author's Medium articles."]


09:53:42 - LiteLLM:ERROR: opik.py:111 - OpikLogger failed to send batch - Client error '403 Forbidden' for url 'https://www.comet.com/opik/api/v1/private/spans/batch'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/litellm/integrations/opik/opik.py", line 102, in _sync_send
    response = self.sync_httpx_client.post(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 372, in post
    raise e
  File "/usr/local/lib/python3.11/dist-packages/litellm/llms/custom_httpx/http_handler.py", line 358, in post
    response.raise_for_status()
  File "/usr/local/lib/python3.11/dist-packages/httpx/_models.py", line 763, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://www.comet.com/op

In [27]:
result

ScoreResult(name='hallucination_metric', value=0.0, reason='[\'The output provides a list of projects mentioned in the context, which is accurate and faithful to the provided context.\', "The output does not introduce any new information beyond what\'s provided in the context.", \'The output does not contradict any information given in the context.\', \'The output does not contradict well-established facts or general knowledge.\', "The output correctly associates the projects with the right entity (the context), which is the author\'s Medium articles."]', metadata=None, scoring_failed=False)

## RAGAS

In [16]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_precision,
    context_recall,
)

retrieved_docs = [x.page_content for x in retriever.invoke(query)]

# Generate the response with JSON formatting
#response = llm_chain.invoke({"question": query, "context": retrieved_docs})


data = {
    "user_input": [query],
    "response": [str(llm_chain.invoke({"question":query,"context":retrieved_docs}))],
    "retrieved_contexts": [retrieved_docs]
}

# Convert the data to a Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Define the metrics you want to evaluate
metrics = [
    faithfulness,
    answer_relevancy,
]


# Evaluate the dataset using the selected metrics
results = evaluate(dataset, metrics,
                   llm=ChatGroq(model="deepseek-r1-distill-llama-70b",),
                   embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),)


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
results

{'faithfulness': 1.0000, 'answer_relevancy': 0.9683}

In [18]:
!pip install giskard[llm] -q

In [19]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in splitted_docs], columns=["text"])
df.head(10)

,text
0,Enhancing RAG Accuracy with Reranking | Medium...
1,search_depth='advanced' ) result...
2,"Retriever for RerankingFor the first approach,..."
3,"related...', 1.6315238412241242)Approach 2: Co..."
4,"Is the Theory of...', 0.20247006687918015)(""Wh..."
5,"if torch.cuda.is_available() else ""cpu"")model...."
6,It then calculates the cosine similarity betwe...
7,helpful. Don’t forget to follow me for more si...
8,AI-Powered Plant Disease Detection | MediumOpe...
9,"As a result, I shifted to leveraging a pre-tra..."


In [20]:
import giskard
from giskard.rag import KnowledgeBase
giskard.llm.set_llm_model("groq/llama-3.2-3b-preview")
giskard.llm.set_embedding_model("mistral/mistral-embed")

knowledge_base = KnowledgeBase(df)

In [23]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=10,
    agent_description="A chatbot answering questions about the Machine Learning School Website",
)

INFO:giskard.rag:Finding topics in the knowledge base.



Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers



/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
INFO:giskard.rag:Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

ERROR:giskard.rag:Encountered error in question generation: 'question'. Skipping.
ERROR:giskard.rag:'question'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/giskard/rag/question_generators/base.py", line 59, in generate_questions
    yield self.generate_single_question(knowledge_base, *args, **kwargs, seed_document=doc)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/giskard/rag/question_generators/simple_questions.py", line 108, in generate_single_question
    question=generated_qa["question"],
             ~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'question'
ERROR:giskard.rag:Encountered error in question generation: 'question'. Skipping.
ERROR:giskard.rag:'question'
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/giskard/rag/question_generators/base.py", line 80, in generate_questions
    yield self._modify_question(question, kn

In [24]:
test_set_df = testset.to_pandas()

In [25]:
def answer_fn(question, history=None):
    return llm_chain.invoke({"question": question,'context':retriever.invoke(query)}).content

In [26]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/7 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/7 [00:00<?, ?it/s]

In [27]:
report